In [8]:
import pandas as pd
from utils import get_best, remove_players, prepare_ratings
from formations import unbeaten_tactics, angeball_v3, perfect_alonso, daly_counter, italian_ko, girona_2324

ratings_raw = pd.read_excel("data/CorinthiansFM24.xlsx", sheet_name="jul2023").sort_values("Highest Role Score", ascending=False)
ratings_raw = ratings_raw.loc[(ratings_raw["Club"] == "Corinthians") & (ratings_raw["Name"] != "Michael")]
sold_out = ["Ivan", "Luiz Mandaca", "Léo Santos", "Giovane", "Rodrigo Varanda", "Vítor Meer", "Léo Mana"]
ratings_raw = remove_players(ratings_raw, sold_out)

squad = [
 "Beto",
 "Caetano",
 "Carlos Miguel",
 "Cássio",
 "Fausto Vera",
#  "Fágner",
 "Gabriel Moscardo",
#  "Giovani",
 "Guilherme Biro",
#  "Gustavo Silva",
#  "Lucas Veríssimo",
 "Léo Natel",
 "Matheus Araújo",
#  "Matheus Bidu",
 "Matheus Donelli",
#  "Matías Rojas",
 "Maycon",
 "Pedro",
#  "Rafael Ramos",
 "Raul Gustavo",
 "Renato Augusto",
 "Roni",
#  "Saúl Salcedo",
 "Wesley",
#  "Yuri Alberto",
 "Ángel Romero"
 ]

try:
    ratings_squad = ratings_raw.query("Name in @squad")
    
except:
    ratings_squad = ratings_raw


# ratings_squad = ratings_raw.query("Name in @squad")
ang_ratings = prepare_ratings(ratings=ratings_squad, formation=angeball_v3)
alonso_ratings = prepare_ratings(ratings=ratings_squad, formation=perfect_alonso)
daly_ratings = prepare_ratings(ratings=ratings_squad, formation=daly_counter)



#### For Angeball v3
ang1, ang1_score = get_best(ang_ratings, angeball_v3, age_constraint=25)
ang1_df = pd.DataFrame(ang1, columns = ["name", "position", "score"]).sort_values("position").reset_index(drop=True)
ang_ratings=remove_players(ang_ratings, [player["name"] for player in ang1])

ang2, ang2_score = get_best(ang_ratings, angeball_v3, age_constraint=25)
ang2_df = pd.DataFrame(ang2, columns = ["name", "position", "score"]).sort_values("position").reset_index(drop=True)
# ang_ratings=remove_players(ang_ratings, [player["name"] for player in ang2])

ang_ratings_yng = ang_ratings.query("Age <= 20 and Age>16")
loan_out, _ = get_best(ang_ratings_yng, angeball_v3)

sell_out = ang_ratings.query("Age >= 23")
limbo = ang_ratings.query("Age <23 and Age > 20")

#### For Daly"s Counter
daly1, daly1_score = get_best(daly_ratings, daly_counter, age_constraint=28)
daly1_df = pd.DataFrame(daly1, columns=["name", "position", "score"]).sort_values("position").reset_index(drop=True)

daly_ratings=remove_players(daly_ratings, [player["name"] for player in daly1])
daly2, daly2_score = get_best(daly_ratings, daly_counter, age_constraint=26)

daly_ratings=remove_players(daly_ratings, [player["name"] for player in daly2])
daly3, daly3_score = get_best(daly_ratings, daly_counter, age_constraint=24)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /root/anaconda3/envs/learn/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d8634e51fbc0410f9e4be1b345e77015-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/d8634e51fbc0410f9e4be1b345e77015-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 1129 RHS
At line 1165 BOUNDS
At line 1302 ENDATA
Problem MODEL has 35 rows, 136 columns and 680 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 141.611 - 0.01 seconds
Cgl0004I processed model has 27 rows, 136 columns (136 integer (136 of which binary)) and 544 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.33333
Cbc0038I Pass   1: suminf.    0.42105 (2) obj. -140.416 iterations 18
Cbc0038I Solution found of -135.6
Cbc0038I Before

In [9]:
daly1_df

,name,position,score
0,Guilherme Biro,afa,12.5
1,Roni,b2bs,13.0
2,Gabriel Moscardo,bpdd,12.8
3,Raul Gustavo,cdd,13.5
4,Renato Augusto,dlfs,12.8
5,Fausto Vera,dlpd,13.1
6,Cássio,gkd,13.1
7,Ángel Romero,iwa,12.6
8,Maycon,wbs,12.6
9,Caetano,wbs,12.1


In [7]:
ratings_raw[ratings_raw["Name"] == "Matheuzinho"]["Club"] = "Flamengo"
ratings_raw.to_excel()

/tmp/ipykernel_26666/3990340801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_raw[ratings_raw["Name"] == "Matheuzinho"]["Club"] = "Flamengo"


In [43]:
ratings_raw[ratings_raw["Name"].isin(["Rodrigo Caio", "Saúl Salcedo", "Igor Rabello"])][["Name", "cdd"]]


,Name,cdd
97,Saúl Salcedo,13.5
91,Rodrigo Caio,13.3
105,Igor Rabello,13.3


In [ ]:
def treat_transfer_value(df):
    # Remove "£" Symbol
    df["Transfer Value"] = df["Transfer Value"].str.replace("£", "")

    # Split Range Values
    df[["lower_bound", "upper_bound"]] = df["Transfer Value"].str.split(" - ", expand=True)

    # Handle Million (M) Values
    df["lower_bound"] = df["lower_bound"].str.replace("M", "e6")
    df["upper_bound"] = df["upper_bound"].str.replace("M", "e6")

    # Handle Thousand (K) Values
    df["lower_bound"] = df["lower_bound"].str.replace("K", "e3")
    df["upper_bound"] = df["upper_bound"].str.replace("K", "e3")

    # Remove commas and convert to numeric
    df["lower_bound"] = df["lower_bound"].str.replace(",", ".")
    df["upper_bound"] = df["upper_bound"].str.replace(",", ".")

    df["lower_bound"] = df["lower_bound"].astype(float)
    df["upper_bound"] = df["upper_bound"].astype(float)

    # Convert to Numeric
    df["lower_bound"] = pd.to_numeric(df["lower_bound"])
    df["upper_bound"] = pd.to_numeric(df["upper_bound"])

    # Create a column for mean between upper and lower bounds
    df["mean_value"] = df.apply(lambda row: row["lower_bound"] if pd.isna(row["upper_bound"]) else (row["lower_bound"] + row["upper_bound"]) / 2, axis=1)

    # Display the formatted DataFrame
    return df

players = treat_transfer_value(ratings_raw)

from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
players_slice = players.query("Name != 'Giovani'")
players_slice["mean_value_scaled"] = scaler.fit_transform(players_slice["mean_value"].values.reshape(-1,1))
players_slice["highest_role_score_scaled"] = scaler.fit_transform(players_slice["Highest Role Score"].values.reshape(-1,1))
players_slice["value_quotient"] = players_slice["highest_role_score_scaled"] / players_slice["mean_value_scaled"]
players_slice["value_quotient_cat"] = pd.cut(players_slice["value_quotient"], bins=[0., 0.25, 0.5, 1., 1.5, 2., 5., np.inf], labels=["vey overpriced", "overpriced", "reasonable_high", "reasonable_low", "underpriced", "very_underpriced", "astronomically_underpriced"])
players_slice.loc[players_slice["Age"]>20].head(30)


/tmp/ipykernel_112099/2359666278.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_slice["mean_value_scaled"] = scaler.fit_transform(players_slice["mean_value"].values.reshape(-1,1))
/tmp/ipykernel_112099/2359666278.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_slice["highest_role_score_scaled"] = scaler.fit_transform(players_slice["Highest Role Score"].values.reshape(-1,1))
/tmp/ipykernel_112099/2359666278.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,Inf,Name,Age,Club,Transfer Value,Wage,Nat,Position,Personality,Media Handling,...,wbs,Highest Role Score,Resulting Role,lower_bound,upper_bound,mean_value,mean_value_scaled,highest_role_score_scaled,value_quotient,value_quotient_cat
10,Hol,Lucas Veríssimo,27,Corinthians,13M - 16M,"£29,500 p/w",BRA,D (C),Fairly Determined,Level-headed,...,12.1,14.4,Central Defender Stopper,13000000.0,16000000.0,14500000.0,1.000000,1.000000,1.000000,reasonable_high
11,Hol,Matías Rojas,27,Corinthians,"2,8M - 4,3M","£25,000 p/w",PAR,"M (RC), AM (RLC)",Fairly Ambitious,Media-friendly,...,12.0,14.0,Trequartista Attack,2800000.0,4300000.0,3550000.0,0.244828,0.935484,3.820990,very_underpriced
5,Hol,Fágner,33,Corinthians,"1,3M - 1,7M","£18,500 p/w",BRA,D/WB (R),Balanced,Media-friendly,...,13.6,13.9,Ball Winning Midfielder Defend,1300000.0,1700000.0,1500000.0,0.103448,0.919355,8.887097,astronomically_underpriced
29,Hol,Yuri Alberto,21,Corinthians,"5,8M - 7,6M","£44,500 p/w",BRA,"AM (RL), ST (C)",Balanced,Media-friendly,...,11.9,13.8,Poacher Attack,5800000.0,7600000.0,6700000.0,0.462069,0.903226,1.954742,underpriced
24,Hol,Bruno Méndez,23,Corinthians,"1,5M - 2,7M","£3,500 p/w",URU,D (RC),Balanced,Level-headed,...,12.7,13.8,Anchor Defend,1500000.0,2700000.0,2100000.0,0.144828,0.903226,6.236559,astronomically_underpriced
3,Hol,Renato Augusto,34,Corinthians,600K - 800K,"£29,500 p/w",BRA,"DM, M (RC), AM (RLC)",Balanced,Level-headed,...,11.7,13.8,Trequartista Attack,600000.0,800000.0,700000.0,0.048276,0.903226,18.709677,astronomically_underpriced
4,Inj,Paulinho,34,Corinthians,425K - 650K,"£28,000 p/w",BRA,"DM, M/AM (C)",Fairly Determined,Level-headed,...,12.9,13.7,Poacher Attack,425000.0,650000.0,537500.0,0.037069,0.887097,23.930983,astronomically_underpriced
15,Inj,Gustavo Silva,25,Corinthians,"2,1M - 3,4M","£9,250 p/w",BRA,"AM (RL), ST (C)",Balanced,Media-friendly,...,12.0,13.7,Winger Support,2100000.0,3400000.0,2750000.0,0.189655,0.887097,4.677419,very_underpriced
20,Hol,Roni,23,Corinthians,"475K - 1,2M","£4,100 p/w",BRA,"DM, M (RC)",Fairly Loyal,Media-friendly,...,12.5,13.4,Ball Winning Midfielder Support,475000.0,1200000.0,837500.0,0.057759,0.838710,14.520944,astronomically_underpriced
22,Hol,Matheus Bidu,23,Corinthians,"700K - 1,8M","£4,100 p/w",BRA,D/WB/M (L),Balanced,Media-friendly,...,13.0,13.4,Ball Winning Midfielder Support,700000.0,1800000.0,1250000.0,0.086207,0.838710,9.729032,astronomically_underpriced


In [ ]:
players_slice.sort_values("mean_value_scaled", ascending=False).head(40)

,Inf,Name,Age,Club,Transfer Value,Wage,Nat,Position,Personality,Media Handling,...,wbs,Highest Role Score,Resulting Role,lower_bound,upper_bound,mean_value,mean_value_scaled,highest_role_score_scaled,value_quotient,value_quotient_cat
10,Hol,Lucas Veríssimo,27,Corinthians,13M - 16M,"£29,500 p/w",BRA,D (C),Fairly Determined,Level-headed,...,12.1,14.4,Central Defender Stopper,13000000.0,16000000.0,14500000.0,1.000000,1.000000,1.000000,reasonable_high
65,Hol,Wesley,17,Corinthians,"9,8M - 12M","£3,400 p/w",BRA,"M/AM (L), ST (C)",Spirited,"Unflappable, Media-friendly",...,11.2,12.6,Winger Attack,9800000.0,12000000.0,10900000.0,0.751724,0.709677,0.944066,reasonable_high
71,Hol,Gabriel Moscardo,17,Corinthians,"9,2M - 12M","£4,100 p/w",BRA,DM,Fairly Determined,Level-headed,...,12.3,13.1,Nononsense Centre Back Cover,9200000.0,12000000.0,10600000.0,0.731034,0.790323,1.081102,reasonable_low
26,Hol,Fausto Vera,22,Corinthians,"8,2M - 9M","£9,750 p/w",ARG,"DM, M (C)",Fairly Ambitious,Media-friendly,...,12.2,13.1,Deep Lying Playmaker Defend,8200000.0,9000000.0,8600000.0,0.593103,0.790323,1.332521,reasonable_low
29,Hol,Yuri Alberto,21,Corinthians,"5,8M - 7,6M","£44,500 p/w",BRA,"AM (RL), ST (C)",Balanced,Media-friendly,...,11.9,13.8,Poacher Attack,5800000.0,7600000.0,6700000.0,0.462069,0.903226,1.954742,underpriced
14,Hol,Maycon,25,Corinthians,"5,2M - 6,6M","£5,750 p/w",BRA,"DM, M (C)",Balanced,"Outspoken, Media-friendly",...,12.6,13.3,Central Midfielder Attack,5200000.0,6600000.0,5900000.0,0.406897,0.822581,2.021597,very_underpriced
76,Hol,Pedro,16,Corinthians,"4,2M - 5,6M",£975 p/w,BRA,AM (RLC),Balanced,Media-friendly,...,11.2,12.2,Inside Forward Attack,4200000.0,5600000.0,4900000.0,0.337931,0.645161,1.909151,underpriced
61,Hol,Breno Bidon,17,Corinthians,"2,6M - 5M",£475 p/w,BRA,M/AM (C),Balanced,Media-friendly,...,10.7,11.5,False Nine Support,2600000.0,5000000.0,3800000.0,0.262069,0.532258,2.030985,very_underpriced
11,Hol,Matías Rojas,27,Corinthians,"2,8M - 4,3M","£25,000 p/w",PAR,"M (RC), AM (RLC)",Fairly Ambitious,Media-friendly,...,12.0,14.0,Trequartista Attack,2800000.0,4300000.0,3550000.0,0.244828,0.935484,3.820990,very_underpriced
32,Hol,Matheus Donelli,20,Corinthians,"2,6M - 4,1M","£1,200 p/w",BRA,GK,Balanced,Media-friendly,...,8.3,12.1,Goalkeeper Defend,2600000.0,4100000.0,3350000.0,0.231034,0.629032,2.722677,very_underpriced
